In [1]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from rdflib import Graph
from ampligraph.utils import save_model, restore_model
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

2023-12-15 16:23:02.081025: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 16:23:02.081047: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 16:23:02.081062: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 16:23:02.084894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.test.is_gpu_available()

2023-12-15 16:23:05.102091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 16:23:05.107591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 16:23:05.107724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

True

In [3]:
g = Graph()
g.parse("../data/music_graph.ttl");

In [4]:
data = [triplet for triplet in g]
len(data)

795348

In [5]:
cols = ["subject", "predicate", "object"]
def remove_prefix(x):
    if type(x) == str:
        return re.sub(".+#", "", x)
    return x
triples_df = pd.DataFrame(data, columns=cols)
for c in cols:
    triples_df[c] = triples_df[c].apply(lambda x: x.toPython())
    triples_df[c] = triples_df[c].apply(remove_prefix) 
triples_df

,subject,predicate,object
0,track_0A3FrkpnqUGzg8E2Kv9z6B,duration,232506
1,track_16rkVc95novr84P1gmSLNC,releaseYear,2002
2,track_0QkMarFV8j9Fq858P0R5y3,tempo,106.492
3,track_14zWV6w5C2bIR5Jbk4Kcqu,loudness,-18.165
4,track_4iejmSjYQpDhTeKs2eNStP,speechiness,0.0473
...,...,...,...
795343,track_0mDg9v4GnAF9wZVuflWL2s,speechiness,0.237
795344,track_2bjrQOQLRjmO09HmhoGzTg,acousticness,0.987
795345,track_0P7qawfOffqLi88i2uIyNT,performedBy,performer_6y1jbhyM2QbpU3Bm8Z8fbe
795346,performer_7pDBRy9uWy1zq5b0uXIABQ,type,Performer


In [6]:
X_train, X_valid = train_test_split_no_unseen(np.array(data), test_size=10000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (785348, 3)
Test set size:  (10000, 3)


In [7]:
model = ScoringBasedEmbeddingModel(
    k=100, eta=1, scoring_type='TransE'
)

2023-12-15 16:24:48.316725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 16:24:48.316892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 16:24:48.317007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = get_loss('multiclass_nll')
regularizer = get_regularizer('LP', {'p': 3, 'lambda': 1e-5})

In [9]:
model.compile(
    optimizer=optim, loss=loss, entity_relation_regularizer=regularizer
)

In [10]:
# Fit the model
model.fit(
    X_train,
    batch_size=32768, # int(X_train.shape[0] / 50),
    epochs=1000,  # Number of training epochs
    verbose=True  # Enable stdout messages
);

Epoch 1/1000


2023-12-15 16:25:01.827840: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd4e82080c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-15 16:25:01.827858: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2023-12-15 16:25:01.830688: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


      2/Unknown - 3s 1s/step - loss: 22739.9688

2023-12-15 16:25:02.057343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-15 16:25:02.135518: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 3s 134ms/step - loss: 22335.2812
Epoch 2/1000
 6/25 [======>.......................] - ETA: 0s - loss: 22234.8184

...

2023-12-15 16:38:12.193507: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4846852865060085533


25/25 [==============================] - 1s 30ms/step - loss: 2142.8801
Epoch 1000/1000
 6/25 [======>.......................] - ETA: 0s - loss: 2142.5554

2023-12-15 16:38:12.977409: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4846852865060085533


25/25 [==============================] - 1s 30ms/step - loss: 2141.3127


2023-12-15 16:38:13.762687: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4846852865060085533


In [11]:
model_filename = "kg_embedding_model"
save_model(model, model_filename)

In [12]:
model = restore_model(model_filename)

Saved model does not include a db file. Skipping.


In [13]:
filter_triples = {'test': np.concatenate((X_train, X_valid))}
ranks = model.evaluate(
    X_valid, use_filter=filter_triples, verbose=True
)

314/314 [==============================] - 135s 431ms/step


2023-12-15 16:46:10.258881: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16827588658910252148
2023-12-15 16:46:10.258919: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15655188065703838059
2023-12-15 16:46:10.258943: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4678047543700124417
2023-12-15 16:46:10.258953: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3833713327968896191
2023-12-15 16:46:10.258961: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 12071121854781629878
2023-12-15 16:46:10.258973: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1354222259333518419
2023-12-15 16:46:10.258982: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

In [28]:
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print(f"MRR: {mrr:.2f}")
print(f"MR: {mr:.2f}")

MRR: 0.22
MR: 5234.97


In [29]:
hits_10 = hits_at_n_score(ranks, n=10)
hits_3 = hits_at_n_score(ranks, n=3)
hits_1 = hits_at_n_score(ranks, n=1)
print(f"Hits@10: {hits_10:.2f}")
print(f"Hits@3: {hits_3:.2f}")
print(f"Hits@1: {hits_1:.2f}")

Hits@10: 0.28
Hits@3: 0.23
Hits@1: 0.19


In [18]:
triples_df[(triples_df["predicate"] == "name") & triples_df["subject"].str.startswith("track")]

,subject,predicate,object
85,track_0dlenAteLFxPm5tL5gPekZ,name,Bad Boys
86,track_3lS5srXPmDwl0Kxgj2V0Ld,name,Expecting
135,track_0CoOGVpjoUlrj9wm4F64XH,name,Kingston
191,track_0rffrBUYsrOpJg52brsUYy,name,What You Sell
247,track_2ExpCKBxiHbgzepERl1t4D,name,Cuando Pase El Temblor
...,...,...,...
795184,track_0QmSEMpnWRrVZ1o9oyjZF3,name,Not The Same
795235,track_4lYuqxULQ8hR82nP5ic8nz,name,Winter Hymn
795310,track_0Fvwx6hiJJhKx32cMGzbBv,name,Rock Wit U (Awww Baby)
795316,track_0HxI6php9EVG5W88jW5EOQ,name,Swing The Heartache


In [21]:
emb = model.get_embeddings(["Bad Boys"])
emb.shape

(1, 100)